# Drift validation

In [20]:
import iris
import numpy as np
import matplotlib.pyplot as plt
import glob

import pandas as pd

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

from iris.experimental.equalise_cubes import equalise_attributes

In [21]:
output_notebook()

Loading BokehJS ...

### Read data

In [4]:
institution = 'NOAA-GFDL'
model = 'GFDL-CM3'
control_location = '/g/data/ua6'  # /g/data/r87/dbi599
historical_location = '/g/data/ua6'

In [5]:
control_files = glob.glob('%s/drstree/CMIP5/GCM/%s/%s/piControl/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_piControl_r1i1p1_*.nc' %(control_location, institution, model, model) )
historical_files = glob.glob('%s/drstree/CMIP5/GCM/%s/%s/historical/mon/ocean/thetao/r1i1p1/thetao_Omon_%s_historical_r1i1p1_*.nc' %(historical_location, institution, model, model) )
dedrifted_historical_files = glob.glob('/g/data/r87/dbi599/drstree/CMIP5/GCM/%s/%s/historical/mon/ocean/thetao/r1i1p1/dedrifted/thetao_Omon_%s_historical_r1i1p1_*.nc' %(institution, model, model) )
coefficient_file = '/g/data/r87/dbi599/drstree/CMIP5/GCM/%s/%s/piControl/mon/ocean/thetao/r1i1p1/thetao-coefficients_Omon_%s_piControl_r1i1p1_all.nc' %(institution, model, model)

In [6]:
# Set constraints
#min_lat, max_lat = [-62.1, -60.9]
#lat_subset = lambda cell: min_lat <= cell <= max_lat
lev_constraint = iris.Constraint(depth=15)
lat_constraint = iris.Constraint(grid_latitude=-63.5)
lon_constraint = iris.Constraint(grid_longitude=70.5)

In [7]:
def read_files(file_list):
    """Read a list of files into an iris Cube"""
    
    cube = iris.load(file_list, 'sea_water_potential_temperature' & lev_constraint & lat_constraint & lon_constraint)
    equalise_attributes(cube)
    cube = cube.concatenate_cube()
    
    return cube

In [8]:
control_cube = read_files(control_files)
historical_cube = read_files(historical_files)
dedrifted_historical_cube = read_files(dedrifted_historical_files)
coefficient_cube = iris.load_cube(coefficient_file, 'Sea Water Potential Temperature' & lev_constraint & lat_constraint & lon_constraint)

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:1139: UserWarning: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warnings.warn(msg)


In [9]:
print control_cube

sea_water_potential_temperature / (K) (time: 6000)
     Dimension coordinates:
          time                             x
     Scalar coordinates:
          depth: 15.0 m, bound=(10.0, 20.0) m
          grid_latitude: -63.5 degrees
          grid_longitude: 70.5 degrees
          latitude: -63.5 degrees, bound=(-64.0, -64.0, -63.0, -63.0) degrees
          longitude: 70.5 degrees, bound=(70.0, 71.0, 71.0, 70.0) degrees
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_GFDL-CM3_piControl_r0i0p0.nc...
          branch_time: 0.0
          comment: GFDL experiment name: CM3Z_Control-1860_D1
GFDL parent experiment name:...
          comments: Note: This model output is presented on the model's tripolar grid. The...
          contact: gfdl.climate.model.info@noaa.gov
          experiment: pre-industrial control
          experiment_id: piControl
          forcing: N/A
          f

In [10]:
print coefficient_cube

Sea Water Potential Temperature / (1) (drift coefficient: 4)
     Dimension coordinates:
          drift coefficient                             x
     Scalar coordinates:
          depth: 15.0 m, bound=(10.0, 20.0) m
          grid_latitude: -63.5 degrees
          grid_longitude: 70.5 degrees
          latitude: -63.5 degrees, bound=(-64.0, -64.0, -63.0, -63.0) degrees
          longitude: 70.5 degrees, bound=(70.0, 71.0, 71.0, 70.0) degrees
     Attributes:
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_GFDL-CM3_piControl_r0i0p0.nc...
          branch_time: 0.0
          comment: GFDL experiment name: CM3Z_Control-1860_D1
GFDL parent experiment name:...
          comments: Note: This model output is presented on the model's tripolar grid. The...
          contact: gfdl.climate.model.info@noaa.gov
          creation_date: 2011-10-12T00:44:11Z
          experiment: pre-industrial control


In [11]:
print coefficient_cube.data

[  2.74396027e+02  -1.92017706e-05   3.36148942e-10  -1.17815200e-15]


### Validate polynomial contruction

In [12]:
def apply_polynomial(x, a, b, c, d):
    """Evaluate cubic polynomial.

    The axis argument is not used but is required for the function to be 
      used with numpy.apply_over_axes 

    """

    result = a + b * x + c * x**2 + d * x**3 
    
    return result 

In [13]:
coeff_a, coeff_b, coeff_c, coeff_d = coefficient_cube.data
my_poly = apply_polynomial(control_cube.coord('time').points,
                           coeff_a, coeff_b, coeff_c, coeff_d)

### Plot

In [14]:
iris.FUTURE.cell_datetime_objects = True

In [16]:
historical_times = historical_cube.coord('time')
historical_dates = [cell.point.strftime("%Y-%m-%d") for cell in historical_times.cells()]  # cell.point.strftime("%Y-%m-%d")
print type(historical_dates[0])
print historical_dates[1:20]

<type 'str'>
['1860-02-15', '1860-03-16', '1860-04-16', '1860-05-16', '1860-06-16', '1860-07-16', '1860-08-16', '1860-09-16', '1860-10-16', '1860-11-16', '1860-12-16', '1861-01-16', '1861-02-15', '1861-03-16', '1861-04-16', '1861-05-16', '1861-06-16', '1861-07-16', '1861-08-16']


In [17]:
df = pd.DataFrame({ 'Date' : historical_dates,
                    'piControl' : historical_cube.data,
                  })

In [18]:
print df

            Date   piControl
0     1860-01-16  277.017914
1     1860-02-15  277.968292
2     1860-03-16  276.023468
3     1860-04-16  274.006256
4     1860-05-16  272.680542
5     1860-06-16  271.812683
6     1860-07-16  271.489227
7     1860-08-16  271.350464
8     1860-09-16  271.369385
9     1860-10-16  271.422882
10    1860-11-16  271.783813
11    1860-12-16  273.956360
12    1861-01-16  276.829346
13    1861-02-15  277.718292
14    1861-03-16  275.768372
15    1861-04-16  273.717010
16    1861-05-16  272.278595
17    1861-06-16  271.592499
18    1861-07-16  271.472626
19    1861-08-16  271.405640
20    1861-09-16  271.681549
21    1861-10-16  271.711060
22    1861-11-16  272.606964
23    1861-12-16  274.766327
24    1862-01-16  277.148895
25    1862-02-15  277.893829
26    1862-03-16  276.980743
27    1862-04-16  275.108490
28    1862-05-16  273.336029
29    1862-06-16  272.381775
...          ...         ...
1722  2003-07-16  272.838379
1723  2003-08-16  273.105194
1724  2003-09-

In [23]:
def datetime(x):
    return np.array(x, dtype=np.datetime64)

p1 = figure(x_axis_type="datetime", title="Sea water potential temperature")
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Temperature'

p1.line(datetime(df['Date']), df['piControl'], color='#A6CEE3', legend='piControl')
p1.legend.location = "top_left"

#output_file("drift.html", title="Drift Check")

show(p1)  # open a browser